# Esercizio 2 - Content2Form

In questo esercizio partendo dalle stesse definizioni date dagli studenti usate nell'esercizio 1, si vuole provare a inferire il miglior synset corrispondente a quelle definizioni.  
Si tratta di **Onomasiologic search**.

### Idea:
- Esattamente come nell'esercizio 1, si salva l'input in un dizionario e si pulisce il dizionario
- Si trovano le parole più frequenti per ogni lista di definizioni relative ad ogni termine (come nell'esercizio 1)
- Si considerano le parole più frequenti come **genus** 
- Si preleva da wordnet synset name, definition e examples per *ogni* **iponimo** di ogni **genus**
- Si sceglie il synset con la definizione più simile alla lista delle definizioni, ovvero con più parole in comune

In [1]:
import pandas as pd
import nltk
from collections import Counter
from nltk.corpus import wordnet as wn

### Leggi il file di input

Salvo le varie definizioni degli studenti in un dizionario con:
- chiave: parola (in ordine Emotion, Person, Revenge, Brick)
- valore: lista di definizioni date dagli studenti

In [2]:
df = pd.read_excel('../data/definizioni.xlsx')
students = df.columns[1:].tolist()
words = df[df.columns[0]].tolist()
def_dict = {}
for definition in words:
    def_dict[definition] = []
for student in students:
    for definition in words:
        def_dict[definition].append(df[student][df[df.columns[0]] == definition].values[0])

# def_dict

### Ripulisci il dictionary delle definizioni

Salvo le definizioni pulite in un dizionario strutturato come il precedente

In [3]:
stopwords = nltk.corpus.stopwords.words('english')
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
lemmatizer = nltk.stem.WordNetLemmatizer()

# Tokenize and lemmatize definitions and remove stopwords and punctuation
def clean_definitions(dict):
    cleaned = dict.copy()
    for key in cleaned:
        definitions = [d for d in cleaned[key] if d == d]
        tmp = []
        for definition in definitions:
            tmp.append([lemmatizer.lemmatize(token.lower()) for token in tokenizer.tokenize(definition) if token.lower() not in stopwords])
        cleaned[key] = tmp
    return cleaned

clean_def_dict = clean_definitions(def_dict)

### Trova i genus

Per ogni parola, calcolo le *n* parole più frequenti nelle definizioni date dagli studenti, usando la funzione `most_common` di `Counter`.  
Queste saranno considerate i **genus**.

In [4]:
# Get the most common words in the definitions of a word
def get_common_words(dict, word, n):
    words = []
    for definition in dict[word]:
        words += definition
    return Counter(words).most_common(n)

for word in clean_def_dict:
    print(word, get_common_words(clean_def_dict, word, 5))

Emotion [('feeling', 13), ('human', 8), ('feel', 8), ('something', 7), ('state', 5)]
Person [('human', 29), ('person', 6), ('living', 4), ('individual', 3), ('certain', 3)]
Revenge [('someone', 14), ('anger', 8), ('feeling', 7), ('action', 6), ('emotion', 6)]
Brick [('used', 24), ('object', 16), ('material', 16), ('construction', 16), ('build', 13)]


### Trova tutti gli iponimi di ogni genus

Per ogni genus, trovo tutti gli iponimi e salvo i loro nomi, definizioni e esempi in una lista ripulita.

In [48]:
# Find all hyponyms for all synsets of a word
def find_hyponyms(word):
    synsets = wn.synsets(word)
    hyponyms = []
    for synset in synsets:
        # hyponyms += synset.hyponyms() # Use this to get only direct hyponyms
        hyponyms += synset.closure(lambda s:s.hyponyms()) # Use this to get all hyponyms
    return set(hyponyms)

# Save hyponyms name, definition and examples in a dictionary for a genus (frequent word)
def get_genus_hypo_dict(word):
    hyponyms = find_hyponyms(word)
    hyponyms_dict = {}
    for hyponym in hyponyms:
        defs = hyponym.definition()
        examples = hyponym.examples()
        tmp = []
        tmp.append(defs)
        for example in examples:
            tmp.append(example)
        hyponyms_dict[hyponym.name()] = tmp

    return hyponyms_dict

# Preprocess hyponyms definition and examples
def clean_hyponyms_dict(dict):
    cleaned = dict.copy()
    for key in cleaned:
        for definition in cleaned[key]:
            cleaned[key] = [lemmatizer.lemmatize(token.lower()) for token in tokenizer.tokenize(definition) if token.lower() not in stopwords]
    return cleaned

# hypo = get_genus_hypo_dict('human')
# cleaned = clean_hyponyms_dict(hypo)
# # print(hypo)
# print(cleaned)

### Trova il synset più simile alle definizioni

Per ogni iponimo, calcolo la similarità tra la lista delle definizioni date dagli studenti e la definizione dell'iponimo.  
La similarità è calcolata come il numero di parole in comune tra le due liste.

In [51]:
# Compute the similarity between the list of definitions of a word and the definition of a hyponym as the number of common words
# def compute_similarity(def_dict, word, hyponym, cleaned_hypo_dict):
#     similarity = 0
#     for definition in def_dict[word]:
#         for word in definition:
#             if word in cleaned_hypo_dict[hyponym]:
#                 # print(f'Definition: {definition}')
#                 # print(f'Hyponym name: {hyponym}')
#                 # print(f'Common word: {word}')
#                 similarity += 1
#     return similarity

def compute_similarity(def_dict, word, hyponym, cleaned_hypo_dict):
    similarity = 0
    set_def_words = set()
    for definition in def_dict[word]:
        for word in definition:
            set_def_words.add(word)
    set_hypo_words = set()
    for word in cleaned_hypo_dict[hyponym]:
        set_hypo_words.add(word)
    similarity = len(set_def_words.intersection(set_hypo_words))
    return similarity

# Find the most similar hyponym computing the similarity for all hyponyms
def find_most_similar_hypo(def_dict, word, cleaned_hypo_dict):
    most_similar = ''
    max_similarity = 0
    for hypo in cleaned_hypo_dict:
        similarity = compute_similarity(def_dict, word, hypo, cleaned_hypo_dict)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar = hypo
    # most_similar = most_similar.split('.')[0]
    return most_similar, max_similarity

# Find the most similar hyponym from all genus (frequent words)
def find_synset(def_dict, word, n):
    freq_words = get_common_words(def_dict, word, n)
    freqs = []
    syn = ''
    max_similarity = 0
    winning_genus = ''
    for freq in freq_words:
        freqs.append(freq[0])
        hypos = get_genus_hypo_dict(freq[0])
        hypo_cleaned = clean_hyponyms_dict(hypos)
        hypo, similarity = find_most_similar_hypo(def_dict, word, hypo_cleaned)
        if similarity > max_similarity:
            max_similarity = similarity
            syn = hypo
            winning_genus = freq[0]
    return syn, max_similarity, freqs, winning_genus

for key in clean_def_dict:
    n_freqs = 3
    syn, similarity, freqs, winning = find_synset(clean_def_dict, key, n_freqs)
    print(f'Word: {key}')
    print(f'{n_freqs} most frequent words: {freqs}')
    print(f'Most similar synset: {syn} with a similarity of {similarity}')
    print(f'Coming from genus: {winning}')
    print(f'{syn}: {wn.synset(syn).definition()}')
    print()

Word: Emotion
3 most frequent words: ['feeling', 'human', 'feel']
Most similar synset: hate.n.01 with a similarity of 4
Coming from genus: feeling
hate.n.01: the emotion of intense dislike; a feeling of dislike so strong that it demands action

Word: Person
3 most frequent words: ['human', 'person', 'living']
Most similar synset: homo_sapiens.n.01 with a similarity of 4
Coming from genus: human
homo_sapiens.n.01: the only surviving hominid; species to which modern man belongs; bipedal primate having language and ability to make and use complex tools; brain volume at least 1400 cc

Word: Revenge
3 most frequent words: ['someone', 'anger', 'feeling']
Most similar synset: motile.n.01 with a similarity of 6
Coming from genus: someone
motile.n.01: one whose prevailing mental imagery takes the form of inner feelings of action

Word: Brick
3 most frequent words: ['used', 'object', 'material']
Most similar synset: brick.n.01 with a similarity of 6
Coming from genus: object
brick.n.01: rectangu